In [1]:
from config import *

from easyric.caas_lite import TiffSpliter
from easyric.objects import Pix4D
from easyric.io import shp, geotiff, plot
from easyric.calculate import geo2raw, geo2tiff

import os
import shapefile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import tifffile



In [7]:
import re
import platform

# Four different Files:
1. ID map -> the link between QGIS shp id & field measure
2. Non-destructive test (West + East) -> field measure (id, 0 45 90 135)
3. Destructive -> field measure weight (very regular)

uav date: "210512", "210514", "210515", "210519", "210520", "210526

In [8]:
user = platform.node()

if user == "DESKTOP-3J8SGRC":
    root = r"Y:\hwang_Pro\data\2021_tanashi_broccoli\03_field_measurement"
elif user == "NERV":
    root = "./input/field_21"

In [9]:
excel = {"root": root,
         "drop": ["x", "X", "根",""],
         "link": {
             "name":"ID Map.xlsx",
             "circle": 2,
             "sheet": [
                 {"name": "West side",
                  "st_row": 1,  # count from 0
                  "ed_row": 143, 
                  "usecol": "B:Y",},
                 {"name": "East side",
                  "st_row": 1,  # count from 0
                  "ed_row": 145,
                  "usecol": "D:AA"},
             ],
         },
         "non-e": {
             "name":"non-destructive test(in East side).xlsx",
             "circle": 5,
             "sheet": [
                 {"name": "20210512",
                  "st_row": 1,  # count from 0
                  "ed_row": 145,
                  "usecol": "F:J,O:S,X:AB,AG:AK"},
                 {"name": "20210515",
                  "st_row": 1,  # count from 0
                  "ed_row": 145,
                  "usecol": "F:J,O:S,X:AB,AG:AK"},
                 {"name": "20210517",
                  "st_row": 1,  # count from 0
                  "ed_row": 145,
                  "usecol": "F:J,O:S,X:AB,AG:AK"},
                 {"name": "20210519",
                  "st_row": 1,  # count from 0
                  "ed_row": 145,
                  "usecol": "F:J,O:S,X:AB,AG:AK"},
                 {"name": "20210520",
                  "st_row": 1,  # count from 0
                  "ed_row": 145,
                  "usecol": "F:J,O:S,X:AB,AG:AK"},
             ]
         },
         "non-w" :{
             "name":"non-destructive test(in West side).xlsx",
             "circle": 5,
             "sheet": [
                 {"name": "20210515",
                  "st_row": 1,  # count from 0
                  "ed_row": 143,
                  "usecol": "I:M,R:V,AB:AF,AI:AM"},
                 {"name": "20210517",
                  "st_row": 1,  # count from 0
                  "ed_row": 143,
                  "usecol": "I:M,R:V,Y:AC,AF:AJ"},
                 {"name": "20210519",
                  "st_row": 1,  # count from 0
                  "ed_row": 143,
                  "usecol": "I:M,R:V,AB:AF,AI:AM"},
                 {"name": "20210520",
                  "st_row": 1,  # count from 0
                  "ed_row": 143,
                  "usecol": "I:M,R:V,AB:AF,AI:AM"},
                 {"name": "20210524",
                  "st_row": 1,  # count from 0
                  "ed_row": 143,
                  "usecol": "I:M,R:V,AB:AF,AI:AM"},
                 {"name": "20210526",
                  "st_row": 1,  # count from 0
                  "ed_row": 143,
                  "usecol": "I:M,R:V,AB:AF,AI:AM"},
             ]
         },
         "des":{
             "name": "destructive test.xlsx",
             "circle": None,
             "sheet": [
                 {"name": "20210514-E(No.1)", 
                  "st_row": 1,
                  "usecol": "B:G"},
                 {"name": "20210514-W(No.1)", 
                  "st_row": 1,
                  "usecol": "B:G"},
                 {"name": "20210517-E(No.2)", 
                  "st_row": 1,
                  "usecol": "B:G"},
                 {"name": "20210517-W(No.2)", 
                  "st_row": 1,
                  "usecol": "B:G"},
                 {"name": "20210519-E(No.3)", 
                  "st_row": 1,
                  "usecol": "B:G"},
                 {"name": "20210519-W(No.3)", 
                  "st_row": 1,
                  "usecol": "B:G"},
                 {"name": "20210520-E(No.4)", 
                  "st_row": 1,
                  "usecol": "B:G"},
                 {"name": "20210520-W(No.4)",
                  "st_row": 1,
                  "usecol": "B:G"},
             ]
         }
        }

# Part 1: Get link id

In [4]:
def read_sheet(file, sheet_id, endrow=True):
    
    file_name = os.path.join(excel["root"], excel[file]["name"])
    if endrow:
        source = pd.read_excel(file_name, 
                              sheet_name=excel[file]["sheet"][sheet_id]["name"], 
                              header=excel[file]["sheet"][sheet_id]["st_row"],
                              usecols=excel[file]["sheet"][sheet_id]["usecol"],
                              nrows=excel[file]["sheet"][sheet_id]["ed_row"],
                              dtype=str)
    else:
        source = pd.read_excel(file_name, 
                              sheet_name=excel[file]["sheet"][sheet_id]["name"], 
                              header=excel[file]["sheet"][sheet_id]["st_row"],
                              usecols=excel[file]["sheet"][sheet_id]["usecol"],
                              dtype=str)
        
    circle = excel[file]["circle"]
    if circle is not None:
        col_name = source.columns.values.tolist()
        
        st = 0
        ed = st+circle
        
        fix_name = col_name[st:ed]
        
        merge = source.iloc[:,st:ed]
        while ed < len(col_name):
            st += circle
            ed += circle
            
            to_merge = source.iloc[:,st:ed]
            
            rename_dict = {i:j for i,j in zip(to_merge.columns.values.tolist(), fix_name)}
            
            merge = pd.concat([merge, to_merge.rename(columns=rename_dict)])
        
        merge = merge.dropna()
        return merge.reset_index(drop=True)
            
    else:
        return source

In [56]:
link_w = read_sheet("link", 0)
link_w = link_w.rename(columns={"No.": "field", "欠：x": "img"})
print(f"remove\n{link_w[link_w.img.str.contains('[a-zA-Z]')]}")
link_w = link_w[~link_w.img.str.contains("[a-zA-Z]")]
link_w = link_w.reset_index(drop=True)
link_w["pos"] = "West"
link_w

remove
       field                                       img
171    25-31                                    x(172)
172    25-32                                    x(173)
245   25-105                                         x
505    23-83                                         x
589    22-25                                         x
650    22-86                                         x
745    21-42                                         x
750    21-47                                         x
1015   19-31                                         x
1030   19-46  花蕾はできていなかったが、葉は存在。\n1とカウントし収穫した。\n重量は0g。
1043   19-59                                         x
1057   19-73                                         x
1076   19-92                                         x
1273    17-8                                         x
1409    16-4                                         x


,field,img,pos
0,26-1,1,West
1,26-2,2,West
2,26-3,3.0,West
3,26-4,4.0,West
4,26-5,5.0,West
...,...,...,...
1665,15-135,1670.0,West
1666,15-136,1671.0,West
1667,15-137,1672.0,West
1668,15-138,1673.0,West


In [57]:
link_e = read_sheet("link", 1)
link_e = link_e.rename(columns={"No.": "field", "ID": "img"})

print(f"remove\n{link_e[link_e.img.str.contains('[a-zA-Z]')]}")
link_e = link_e[~link_e.img.str.contains("[a-zA-Z]")]
link_e = link_e.reset_index(drop=True)
link_e["pos"] = "East"
link_e

remove
           field           img
43         12-44       x(1993)
56         12-57       x(2006)
157        11-16             x
203        11-62             x
218        11-77             x
224        11-83             x
372        10-91             x
580         8-16             x
610   8-46(根切り虫)       x(2553)
668        8-104             x
890         6-45             X
929         6-84             x
982        6-137       x(2923)
1028        5-40  x(折れた)(2968)
1042        5-54  x(折れた)(2982)
1050        5-62             x
1104       5-116             x
1105       5-117             x
1106       5-118             x
1164        4-37             x
1178        4-51             x
1300        3-29             x
1306        3-35             x
1308        3-37             x
1389       3-118             x
1442        2-30             x
1453        2-41             x
1486        2-74             x
1509        2-97             x
1662       1-108             x


,field,img,pos
0,12-1,1950.0,East
1,12-2,1951.0,East
2,12-3,1952.0,East
3,12-4,1953.0,East
4,12-5,1954.0,East
...,...,...,...
1663,1-139,3618.0,East
1664,1-140,3619.0,East
1665,1-141,3620.0,East
1666,1-142,3621.0,East


In [58]:
link = pd.concat([link_w, link_e])
link = link.astype({"field":str, "pos": str, "img": float})
link = link.astype({"field":str, "pos": str, "img": int})
link

,field,img,pos
0,26-1,1,West
1,26-2,2,West
2,26-3,3,West
3,26-4,4,West
4,26-5,5,West
...,...,...,...
1663,1-139,3618,East
1664,1-140,3619,East
1665,1-141,3620,East
1666,1-142,3621,East


In [60]:
link.to_csv("out/link_table.csv", index=False)

# Part 2: Get Non-destructive

In [11]:
def clean_data(df, rename_dict, find_col, dtype_dict, insert_col=None):
    print(df.columns.values)
    df = df.rename(columns=rename_dict)
    idx = df[find_col].str.contains('[a-zA-Z]')
    print(f"remove\n{df[idx]}")
    df = df[~idx]
    df = df.reset_index(drop=True)
    
    if insert_col is not None:
        for k, v in insert_col.items():
            df[k] = v
    
    return df

In [83]:
non_e = read_sheet("non-e", 0)
non_e = clean_data(non_e, {"No..1": "field"}, find_col=90, 
                   dtype_dict={"field":str, 0:float, 90:float, 45:float, 135:float},
                   insert_col={"date": "20210506", "pos": "East"})
non_e

remove
          field  0 90 45 135
65         8-16  x  x  x   x
95   8-46(根切り虫)  x  x  x   x
131       5-116  x  x  x   x
132       5-117  x  x  x   x
133       5-118  x  x  x   x
184        2-30  x  x  x   x
195        2-41  x  x  x   x


,field,0,90,45,135,date,pos
0,11-91,5.4,5.4,5.4,5.4,20210506,East
1,11-92,8.3,7.6,8.6,8.1,20210506,East
2,11-93,8.5,8.8,9.3,9.1,20210506,East
3,11-94,7.1,6.8,7.4,7.4,20210506,East
4,11-95,9.1,9.7,9.7,9.8,20210506,East
...,...,...,...,...,...,...,...
196,2-49,10,10.3,10,9.2,20210506,East
197,2-50,8.4,8.5,8.1,8.3,20210506,East
198,2-51,8.3,9.0,8.8,8.3,20210506,East
199,2-52,9.6,10.7,9,10.6,20210506,East


In [87]:
non_list = []

for i, sheet_e in enumerate(excel["non-e"]["sheet"]):
    date = sheet_e["name"]
    
    non_e = read_sheet("non-e", i)
    non_e = clean_data(non_e, {"No..1": "field"}, find_col=90, 
                       dtype_dict={"field":str, 0:float, 90:float, 45:float, 135:float},
                       insert_col={"date": date, "pos": "East"})
    non_list.append(non_e)
    
    
for i, sheet_w in enumerate(excel["non-w"]["sheet"]):
    date = sheet_w["name"]
    non_w = read_sheet("non-w", i)
    non_w = clean_data(non_w, {"No..2": "field", '0.1':0, '90.1':90, '45.1':45, '135.1':135}, find_col=90, 
                       dtype_dict={"field":str, 0:float, 90:float, 45:float, 135:float},
                       insert_col={"date": date, "pos": "West"})
    non_list.append(non_w)
    
non = pd.concat(non_list)
non

['No..1' 0 90 45 135]
remove
          field  0 90 45 135
65         8-16  x  x  x   x
95   8-46(根切り虫)  x  x  x   x
131       5-116  x  x  x   x
132       5-117  x  x  x   x
133       5-118  x  x  x   x
184        2-30  x  x  x   x
195        2-41  x  x  x   x
['No..1' 0 90 45 135]
remove
          field  0 90 45 135
65         8-16  x  x  x   x
95   8-46(根切り虫)  x  x  x   x
131       5-116  x  x  x   x
132       5-117  x  x  x   x
133       5-118  x  x  x   x
184        2-30  x  x  x   x
195        2-41  x  x  x   x
['No..1' 0 90 45 135]
remove
          field  0 90 45 135
65         8-16  x  x  x   x
95   8-46(根切り虫)  x  x  x   x
131       5-116  x  x  x   x
132       5-117  x  x  x   x
133       5-118  x  x  x   x
184        2-30  x  x  x   x
195        2-41  x  x  x   x
['No..1' 0 90 45 135]
remove
          field  0 90 45 135
65         8-16  x  x  x   x
95   8-46(根切り虫)  x  x  x   X
131       5-116  x  x  x   x
132       5-117  x  x  x   x
133       5-118  x  x  x   x
184        2-3

,field,0,90,45,135,date,pos
0,11-91,5.4,5.4,5.4,5.4,20210512,East
1,11-92,8.3,7.6,8.6,8.1,20210512,East
2,11-93,8.5,8.8,9.3,9.1,20210512,East
3,11-94,7.1,6.8,7.4,7.4,20210512,East
4,11-95,9.1,9.7,9.7,9.8,20210512,East
...,...,...,...,...,...,...,...
45,19-138,20.8,18.8,21.1,19.2,20210526,West
46,19-139,20.2,19,20,19.4,20210526,West
47,19-140,19.9,21.1,19.9,19.1,20210526,West
48,19-141,20.9,20.9,18,19.5,20210526,West


In [89]:
non = non[["date", "pos", "field", 0, 90, 45, 135]]
non = non.sort_values(by=['date', 'field'])
non

,date,pos,field,0,90,45,135
9,20210512,East,11-100,7.9,7.1,8.2,7.7
10,20210512,East,11-101,7.9,8.0,7.8,8.8
11,20210512,East,11-102,10.2,9.2,10.1,10.3
12,20210512,East,11-103,9.6,10.5,10,10.5
13,20210512,East,11-104,9.6,9.8,9.8,9.8
...,...,...,...,...,...,...,...
2,20210526,West,19-95,9.4,13.7,10.7,10.5
3,20210526,West,19-96,19.2,20.4,18.7,20.6
4,20210526,West,19-97,19,20,19.6,18.9
5,20210526,West,19-98,17.6,19.8,19,19.1


In [90]:
non.to_csv("out/non_destructive.csv", index=False)

# Part 3: Get Destructive

In [12]:
des_e = read_sheet("des", 0, endrow=False)
des_e = clean_data(des_e, {"Unnamed: 1": "field", "0° (mm)":0, "90° (mm)":90, "45° (mm)":45, "135° (mm)":135, "Weight (g)": "weight_g"}, 
                   find_col=0, 
                   dtype_dict={"field":str, 0:int, 90:int, 45:int, 135:int, "weight_g":float},
                   insert_col={"date": "20210506", "pos": "East"})
des_e

['Unnamed: 1' '0° (mm)' '90° (mm)' '45° (mm)' '135° (mm)' 'Weight (g)']
remove
Empty DataFrame
Columns: [field, 0, 90, 45, 135, weight_g]
Index: []


,field,0,90,45,135,weight_g,date,pos
0,4-75,91,99,88,96,240,20210506,East
1,4-76,136,145,138,137,390,20210506,East
2,4-77,102,108,108,104,250,20210506,East
3,4-78,123,127,135,127,335,20210506,East
4,4-79,107,114,108,117,275,20210506,East
...,...,...,...,...,...,...,...,...
65,4-140,89,88,93,89,205,20210506,East
66,4-141,85,103,92,91,220,20210506,East
67,4-142,102,104,98,104,285,20210506,East
68,4-143,77,85,86,87,215,20210506,East


In [13]:
des_list = []

pos_dict = {'W':"West", "E":"East"}

for i, sheet_d in enumerate(excel["des"]["sheet"]):
    sheet_name = sheet_d["name"]
    date, pos, _ = re.split(f"\-|\(", sheet_name)

    des = read_sheet("des", i, endrow=False)
    des = clean_data(des, {"Unnamed: 1": "field", "0° (mm)":0, "90° (mm)":90, "45° (mm)":45, "135° (mm)":135, "Weight (g)": "weight_g"}, 
                     find_col=0, 
                     dtype_dict={"field":str, 0:int, 90:int, 45:int, 135:int, "weight_g":float},
                     insert_col={"date": date, "pos": pos_dict[pos]})

    des_list.append(des)
    
des = pd.concat(des_list)
des

['Unnamed: 1' '0° (mm)' '90° (mm)' '45° (mm)' '135° (mm)' 'Weight (g)']
remove
Empty DataFrame
Columns: [field, 0, 90, 45, 135, weight_g]
Index: []
['Unnamed: 1' '0° (mm)' '90° (mm)' '45° (mm)' '135° (mm)' 'Weight (g)']
remove
    field  0 90 45 135 weight_g
25  17-95  x  x  x   x        x
['Unnamed: 1' '0° (mm)' '90° (mm)' '45° (mm)' '135° (mm)' 'Weight (g)']
remove
   field              0           90            45           135  \
13  6-14  A: 89, B: 112  A: 89, B:81  A: 90, B: 87  A: 85, B: 86   
44  6-45              x            x             x             x   
45  6-46              x            x             x             x   
46  6-47              x            x             x             x   
47  6-48              x            x             x             x   

          weight_g  
13  A: 110, B: 110  
44               x  
45               x  
46               x  
47               x  
['Unnamed: 1' '0° (mm)' '90° (mm)' '45° (mm)' '135° (mm)' 'Weight (g)']
remove
    field  0 90 

,field,0,90,45,135,weight_g,date,pos
0,4-75,91,99,88,96,240,20210514,East
1,4-76,136,145,138,137,390,20210514,East
2,4-77,102,108,108,104,250,20210514,East
3,4-78,123,127,135,127,335,20210514,East
4,4-79,107,114,108,117,275,20210514,East
...,...,...,...,...,...,...,...,...
65,23-66,123,130,125,120,290,20210520,West
66,23-67,165,152,149,154,450,20210520,West
67,23-68,134,140,137,138,380,20210520,West
68,23-69,113,100,100,111,230,20210520,West


In [14]:
des = des[["date", "pos", "field", 0, 90, 45, 135, "weight_g"]]
des = des.sort_values(by=['date', 'field'])
des

,date,pos,field,0,90,45,135,weight_g
29,20210514,West,17-100,96,107,102,109,240
30,20210514,West,17-101,75,74,69,71,190
31,20210514,West,17-102,76,81,72,82,230
32,20210514,West,17-103,73,82,80,79,200
33,20210514,West,17-104,109,109,110,107,275
...,...,...,...,...,...,...,...,...
68,20210520,West,23-69,113,100,100,111,230
6,20210520,West,23-7,142,140,151,140,460
69,20210520,West,23-70,139,131,140,137,320
7,20210520,West,23-8,149,135,137,143,400


In [15]:
des = des.drop_duplicates()
des

,date,pos,field,0,90,45,135,weight_g
29,20210514,West,17-100,96,107,102,109,240
30,20210514,West,17-101,75,74,69,71,190
31,20210514,West,17-102,76,81,72,82,230
32,20210514,West,17-103,73,82,80,79,200
33,20210514,West,17-104,109,109,110,107,275
...,...,...,...,...,...,...,...,...
68,20210520,West,23-69,113,100,100,111,230
6,20210520,West,23-7,142,140,151,140,460
69,20210520,West,23-70,139,131,140,137,320
7,20210520,West,23-8,149,135,137,143,400


In [16]:
des.to_csv("out/destructive.csv", index=False)